In [4]:
import pandas as pd
import requests
import json
from config import api_key 
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress


In [30]:
#Load CSV Files with movie data
imdb_file = pd.read_csv("Resource/IMDb_All_Genres_etf_clean1.csv")
imdb_clean = imdb_file.drop(columns = ["Censor", "main_genre", "side_genre", "Runtime(Mins)"])
imdb_clean = imdb_clean.rename(columns = {"Rating": "imdb_score"})
imdb_clean.columns

Index(['Movie_Title', 'Year', 'Director', 'Actors', 'imdb_score',
       'Total_Gross'],
      dtype='object')

In [28]:
#Load CSV Files with movie data
metadata_file = pd.read_csv("Resource/movie_metadata.csv")
metadata_clean = metadata_file.drop(columns = ["color", "num_critic_for_reviews", "duration", "director_facebook_likes", 
                             "actor_3_facebook_likes", "actor_1_facebook_likes", "genres", "num_voted_users", 
                             "cast_total_facebook_likes", "facenumber_in_poster", "plot_keywords", "movie_imdb_link", 
                             "num_user_for_reviews", "language", "country", "content_rating", "aspect_ratio", "movie_facebook_likes",
                                              "actor_2_facebook_likes"])
metadata_clean = metadata_clean.rename(columns = {"movie_title": "Movie_Title", "title_year": "Year"})
metadata_clean.columns

Index(['director_name', 'actor_2_name', 'gross', 'actor_1_name', 'Movie_Title',
       'actor_3_name', 'budget', 'Year', 'imdb_score'],
      dtype='object')

In [29]:
#Load CSV Files with movie data
top1000_file = pd.read_csv("Resource/Top_1000_Highest_Grossing_Movies_Of_All_Time.csv")
top1000_file = top1000_file.drop(columns = ["Genre", "Movie Rating", "Duration", "Votes", "Logline"])
top1000_file = top1000_file.rename(columns = {"Movie Title": "Movie_Title", "Year of Realease": "Year"})
top1000_file.columns

Index(['Movie_Title', 'Year', 'Gross', 'Worldwide LT Gross', 'Metascore'], dtype='object')